# Load Initial Libraries Required - PART ONE

In [1]:
import pandas as pd
import numpy as np
import requests

from bs4 import BeautifulSoup

# Load URL

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_df = pd.read_html(url, header=0)[0]


# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
toronto_df.drop(toronto_df[toronto_df['Borough'] == "Not assigned"].index, axis=0, inplace=True)

toronto_df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table

In [9]:
toronto_df = toronto_df.groupby("Postcode").agg(lambda x:','.join(set(x)))

In [10]:
toronto_df.head(20)

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"Ionview,East Birchmount Park,Kennedy Park"
M1L,Scarborough,"Golden Mile,Oakridge,Clairlea"
M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [13]:
toronto_df.loc[toronto_df['Neighbourhood']=="Not assigned",'Neighbourhood']= \
toronto_df.loc[toronto_df['Neighbourhood']=="Not assigned",'Borough']

In [14]:
toronto_df.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
M1E,Scarborough,"Morningside,West Hill,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


_Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making._

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [18]:
print ("no. rows= ", toronto_df.shape[0])

no. rows=  103


# Toronto Coordinates (Loading additional libraries) - PART TWO

In [ ]:
from geopy.geocoders import Nominatim

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library


!wget -q -O 'toronto_data.json' http://cocl.us/Geospatial_data

print('Libraries imported.')
print('Data downloaded!')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

Data downloaded!


In [17]:
toronto_coordinates = pd.read_csv("https://cocl.us/Geospatial_data")
toronto_df['Latitude'] = toronto_coordinates ['Latitude'].values
toronto_df['Longitude']= toronto_coordinates ['Longitude'].values
toronto_df.head()

,Borough,Neighbourhood,Latitude,Longitude
Postcode,,,,
M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [27]:
# define the dataframe columns
column_names = ['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="foursquare_agent") 
location = geolocator.geocode(address) 
latitude = location.latitude 
longitude = location.longitude 
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.653963, -79.387207.


In [14]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' 
 
toronto_df = pd.DataFrame(df) 
toronto_df


,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"West Hill,Morningside,Guildwood"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest"


Data downloaded!


NameError: name 'json' is not defined

In [12]:
toronto_df = df[ df.Borough.str.contains('Toronto') ]
tororonto_df.reset_index(inplace=True)
tororonto_df

NameError: name 'tororonto_df' is not defined